In [2]:
import numpy as np
from sklearn import preprocessing

# extract the dataset from audioApp

In [3]:
raw_csv_data = np.loadtxt('Audiobooks_data.csv',delimiter=',')


In [4]:
unscaled_inputs_all = raw_csv_data[:,1:-1]
unscaled_inputs_all


array([[2.160e+03, 2.160e+03, 1.013e+01, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [1.404e+03, 2.808e+03, 6.660e+00, ..., 0.000e+00, 0.000e+00,
        1.820e+02],
       [3.240e+02, 3.240e+02, 1.013e+01, ..., 0.000e+00, 1.000e+00,
        3.340e+02],
       ...,
       [1.080e+03, 1.080e+03, 6.550e+00, ..., 0.000e+00, 0.000e+00,
        2.900e+01],
       [2.160e+03, 2.160e+03, 6.140e+00, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [1.620e+03, 1.620e+03, 5.330e+00, ..., 0.000e+00, 0.000e+00,
        9.000e+01]])

In [5]:
targets_all = raw_csv_data[:,-1]
targets_all

array([1., 1., 1., ..., 0., 0., 0.])

## Now We Have to Balance the Dataset for equal Number of Targets

In [6]:
num_one_targets = int(np.sum(targets_all))
zero_targets_counter = 0
indices_to_remove = []
for i in range(targets_all.shape[0]):
    if targets_all[i] == 0:
        zero_targets_counter += 1
        if zero_targets_counter > num_one_targets:
            indices_to_remove.append(i)
unscaled_inputs_equal_priors = np.delete(unscaled_inputs_all, indices_to_remove, axis=0)
targets_equal_priors = np.delete(targets_all, indices_to_remove, axis=0)            
            
            
            
            

# Standardize the inputs using preprocessing

In [7]:
scaled_inputs = preprocessing.scale(unscaled_inputs_equal_priors)
scaled_inputs


array([[ 1.18956512,  0.36398846,  0.67728889, ..., -0.8635056 ,
        -0.20536617, -0.77240946],
       [-0.33022754,  1.10843845, -0.08841391, ..., -0.8635056 ,
        -0.20536617,  1.16499791],
       [-2.50135991, -1.74528653,  0.67728889, ..., -0.8635056 ,
         2.23179102,  2.78305242],
       ...,
       [ 1.18956512,  0.36398846,  0.67728889, ..., -0.20129479,
        -0.20536617, -0.62337812],
       [ 1.18956512,  0.36398846,  0.27347444, ..., -0.20129479,
        -0.20536617,  0.21758442],
       [ 1.18956512,  0.36398846,  0.20727535, ..., -0.20129479,
        -0.20536617, -0.51692717]])

## shuffle the data to be best as possible

In [8]:
shuffled_indices = np.arange(scaled_inputs.shape[0])

np.random.shuffle(shuffled_indices)

# Use the shuffled indices to shuffle the inputs and targets
shuffled_inputs = scaled_inputs[shuffled_indices]
shuffled_targets = targets_equal_priors[shuffled_indices]
shuffled_indices

array([ 527,  985, 4399, ...,  950, 1934,  734])

## split the dataset into Train,validation and Test in the ratio of 8:1:1

In [9]:
samples_count = shuffled_inputs.shape[0]

# Count the samples in each subset, assuming we want 80-10-10 distribution of training, validation, and test.
# Naturally, the numbers are integers.
train_samples_count = int(0.8 * samples_count)
validation_samples_count = int(0.1 * samples_count)

# The 'test' dataset contains all remaining data.
test_samples_count = samples_count - train_samples_count - validation_samples_count

# Create variables that record the inputs and targets for training
# In our shuffled dataset, they are the first "train_samples_count" observations
train_inputs = shuffled_inputs[:train_samples_count]
train_targets = shuffled_targets[:train_samples_count]

# Create variables that record the inputs and targets for validation.
# They are the next "validation_samples_count" observations, folllowing the "train_samples_count" we already assigned
validation_inputs = shuffled_inputs[train_samples_count:train_samples_count+validation_samples_count]
validation_targets = shuffled_targets[train_samples_count:train_samples_count+validation_samples_count]

# Create variables that record the inputs and targets for test.
# They are everything that is remaining.
test_inputs = shuffled_inputs[train_samples_count+validation_samples_count:]
test_targets = shuffled_targets[train_samples_count+validation_samples_count:]

# We balanced our dataset to be 50-50 (for targets 0 and 1), but the training, validation, and test were 
# taken from a shuffled dataset. Check if they are balanced, too. Note that each time you rerun this code, 
# you will get different values, as each time they are shuffled randomly.
# Normally you preprocess ONCE, so you need not rerun this code once it is done.
# If you rerun this whole sheet, the npzs will be overwritten with your newly preprocessed data.

# Print the number of targets that are 1s, the total number of samples, and the proportion for training, validation, and test.
print(np.sum(train_targets), train_samples_count, np.sum(train_targets) / train_samples_count)
print(np.sum(validation_targets), validation_samples_count, np.sum(validation_targets) / validation_samples_count)
print(np.sum(test_targets), test_samples_count, np.sum(test_targets) / test_samples_count)

1800.0 3579 0.5029337803855826
208.0 447 0.465324384787472
229.0 448 0.5111607142857143


## Save the dataset for further used in .npz file

In [10]:
np.savez('Audiobooks_data_train', inputs=train_inputs, targets=train_targets)
np.savez('Audiobooks_data_validation', inputs=validation_inputs, targets=validation_targets)
np.savez('Audiobooks_data_test', inputs=test_inputs, targets=test_targets)